In [1]:
# Generates 2D and 3D mesh from a function or topography from a file/function
# Organic layering have constant thickness
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import sys, os

sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools','meshing_ats'))
import meshing_ats

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal
import math

In [14]:
topography_from_Excel = True
topography_from_function1D = False
topography_from_function2D = False

In [15]:
# Plot the topography here so make sure data is read correctly
# if topography is provided through Excel sheet
if (topography_from_Excel):
    xl =pd.ExcelFile('Topo_Profiles_Barrow.xlsx')
    xl.sheet_names
    data = xl.parse(sheet_name='Transect B, Area C')
    keys = data.keys()
    plt.plot(data['Distance_m'], data['Z'])
    plt.ylabel('Elevation')
    plt.title('Topography, Polygon Area C')
    plt.xlabel('Distance [m] (trough-to-trough)')
    plt.show()
    
# To get topography from a 1D function, define it here, e.g., z = sine(x)
if(topography_from_function1D):
    A = 0.15
    k1 = math.pi - 100
    m = .01
    Xf_1d = np.linspace(0,100,101)
    Zf_1d = A*np.sin(k1*Xf) + m*Xf
    plt.plot(Xf,Zf)
    
# To get topography from a 2D function, define it here, e.g., z = sine(x,y)
if(topography_from_function2D):
    A = 0.15
    k1 = math.pi
    k2 = math.pi
    m = 0.1
    Xf_2d = np.linspace(0,2,101)
    Yf_2d = np.linspace(0,2,101)
    [Xf,Yf] = np.meshgrid(Xf_2d,Yf_2d)

    Zf_2d = A*np.sin(k1*Xf)*np.sin(k2*Yf) + m*Xf

    plt.contourf(Xf,Yf,Zf_2d)

In [53]:
# smooth the topographic data'

def surface1D_topography():
    assert topography_from_Excel or topography_from_function1D
    global x 
    global z 
    global z_smooth
    x_index = []
    if topography_from_Excel:
        x = np.array(data['Distance_m'])
        z = np.array(data['Z'])
        x_index = [0, 10, 25, len(x)-30] # indices for smoothing interval
        l = x_index[3]
        print l
    elif (topography_from_function1D):
        x = Xf_1d
        z = Zf_1d
        x_index = [0, 9, 26, len(x)-1] # indices for smoothing interval
    
    smoothed= True
    
    z_smooth = z
    if (smoothed):
        x1 = [x[x_index[0]], x[x_index[1]]]
        z1 = [z[x_index[0]], z[x_index[1]]]

        x2 = [x[x_index[1]], x[x_index[2]], x[x_index[3]]]
        z2 = [z[x_index[1]], z[x_index[2]], z[x_index[3]]]

        cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
        cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')


        z_smooth = np.concatenate((cs1(x[:11]), cs2(x[11:l])))

    z_smooth = np.array([round(s,4) for s in z_smooth])
    x = x[:l]
    plt.plot(x[:l], z_smooth, '-.', color = 'k')
    plt.plot(x1,z1,'*', color='c')
    plt.plot(x2,z2,'*', color='c')

    plt.ylabel('Elevation [m]')
    plt.xlabel('Distance [m]')
    plt.title('Mesh, Polygon Area C')
    plt.show()


In [54]:
surface1D_topography()

47


In [55]:
print x
print z_smooth

[ 0.          0.27521822  0.55043644  0.82565466  1.10087288  1.37609111
  1.65130933  1.92652755  2.20174577  2.47696399  2.75218221  3.02740043
  3.30261865  3.57783688  3.8530551   4.12827332  4.40349154  4.67870976
  4.95392798  5.2291462   5.50436442  5.77958265  6.05480087  6.33001909
  6.60523731  6.88045553  7.15567375  7.43089197  7.70611019  7.98132841
  8.25654664  8.53176486  8.80698308  9.0822013   9.35741952  9.63263774
  9.90785596 10.18307418 10.45829241 10.73351063 11.00872885 11.28394707
 11.55916529 11.83438351 12.10960173 12.38481995 12.66003817]
[4.83   4.835  4.8487 4.8689 4.8934 4.92   4.9466 4.9711 4.9913 5.005
 5.01   5.0096 5.0084 5.0064 5.0038 5.0006 4.9968 4.9926 4.988  4.983
 4.9778 4.9724 4.9668 4.9612 4.9556 4.95   4.9446 4.9393 4.9341 4.9292
 4.9244 4.9198 4.9153 4.9111 4.9071 4.9034 4.8999 4.8966 4.8936 4.8909
 4.8884 4.8862 4.8844 4.8828 4.8816 4.8807 4.8802]


In [56]:
reload(meshing_ats)
if (topography_from_function1D or topography_from_Excel):
    m2 = meshing_ats.Mesh2D.from_Transect(x,z_smooth)
elif(topography_from_function2D):
    m2 = meshing_ats.Mesh2D.from_2DSurface(Xf_2d,Yf_2d,Zf_2d) #1D x, 1D y, and 2D Z
m2.plot()


(94, 3) 2


In [14]:
outfile = "/Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/"
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []

for i in range(1):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1001)
    z = z + 0.02
    Z.append(z)
print ('Moss ', z)

for i in range(4):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1002)
    z = z + 0.02
    Z.append(z)
print ('Peat ', z)

for i in range(15): 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + 0.02
    Z.append(z)
print ('Upper mineral ', z)

dz = .02
for i in range(35):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print ('Ice rich', z)

for i in range(25):
    dz *= 1.12
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print (z)


layer_types.append('snapped')
layer_data.append(-45.0) # bottom location
layer_ncells.append(1)
layer_mat_ids.append(1003)

m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)


m3.write_exodus(outfile + "areaC-3layers-om10cm-l.exo")


('Moss ', 0.02)
('Peat ', 0.1)
('Upper mineral ', 0.4000000000000001)
('Ice rich', 3.716409519999923)
41.2555869471

You are using exodus.py v 1.05 (beta-cmake), a python wrapper of some of the exodus II library.

Copyright (c) 2013, 2014, 2015, 2016, 2017 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.  

Opening exodus file: /Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/areaC-3layers-om10cm-l0000.exo
Closing exodus file: /Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/areaC-3layers-om10cm-l0000.exo
